In [3]:
from gensim.models import KeyedVectors
# Load the Word2Vec model (this may take a few minutes)
model_path = 'GoogleNews-vectors-negative300.bin'
word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)

result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = result[0]  # look at the first match
print(f"{most_similar_key}: {similarity:.4f}")


queen: 0.9314


In [8]:
result = word_vectors.most_similar_cosmul(positive=['plant', 'desert'], negative=['water'])
for val in result[0:3]:
    most_similar_key, similarity = val
    print(f"{most_similar_key}: {similarity:.4f}")

Desert_Proving_Ground: 0.8663
factory: 0.8367
Ramos_Arizpe: 0.8350


In [40]:
import gensim.downloader
from gensim.models import KeyedVectors
import nltk
from nltk.corpus import words
# Load the Word2Vec model (this may take a few minutes)
# glove_tw25_vectors = gensim.downloader.load('glove-twitter-25')
# glove_wiki50_vectors = gensim.downloader.load('glove-wiki-gigaword-50')
glove_wiki300_vectors = gensim.downloader.load('glove-wiki-gigaword-300')


[==================================================] 100.0% 376.1/376.1MB downloaded


2024-02-12 10:31:46,836 : INFO : glove-wiki-gigaword-300 downloaded
2024-02-12 10:31:46,840 : INFO : loading projection weights from /Users/Alex/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2024-02-12 10:32:42,474 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (400000, 300) matrix of type float32 from /Users/Alex/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-02-12T10:32:42.474579', 'gensim': '4.3.2', 'python': '3.11.6 (main, Oct  2 2023, 13:45:54) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.3.1-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [143]:
import gensim.downloader
from gensim.models import KeyedVectors
import nltk
from nltk.corpus import words
model = gensim.downloader.load('glove-wiki-gigaword-300')
nltk.download('words')
english_words = set(words.words())

# Filter the model's vocabulary
filtered_vocab = {
    word: model[word] for word in model.key_to_index
    if word in english_words
}
new_kv = KeyedVectors(vector_size=model.vector_size)

# Prepare lists of keys (words) and their vectors
keys = list(filtered_vocab.keys())
vectors = [filtered_vocab[word] for word in keys]
new_kv.sort_by_descending_frequency()
# Add all vectors in one batch
new_kv.add_vectors(keys, vectors)
new_kv.most_similar(positive='man', negative = 'woman', restrict_vocab=50000)


[nltk_data] Downloading package words to /Users/Alex/nltk_data...
[nltk_data]   Package words is already up-to-date!


[('beefy', 0.2662779688835144),
 ('burly', 0.2654513120651245),
 ('brother', 0.26382869482040405),
 ('guy', 0.2579931318759918),
 ('himself', 0.24928799271583557),
 ('hero', 0.24625438451766968),
 ('mismeasure', 0.24512620270252228),
 ('jimmy', 0.2413175106048584),
 ('arsenal', 0.23860912024974823),
 ('mustache', 0.23555244505405426)]

In [137]:
import nltk
from nltk.corpus import words, brown
from nltk import FreqDist
from gensim.models import KeyedVectors

model = glove_wiki300_vectors

# Download and prepare the list of English words and Brown frequency list
nltk.download('words')
nltk.download('brown')
english_words = set(words.words())
frequency_list = FreqDist(i.lower() for i in brown.words())

# Filter the model's vocabulary
filtered_vocab = {
    word: {"vector": model[word], "frequency": frequency_list[word]} for word in model.key_to_index
    if word in english_words and word in frequency_list
}
new_kv = KeyedVectors(vector_size=model.vector_size)

# Prepare lists of keys (words) and their vectors
keys = list(filtered_vocab.keys())
vectors = [filtered_vocab[word]["vector"] for word in keys]
frequencies = [filtered_vocab[word]["frequency"] for word in keys]
new_kv.sort_by_descending_frequency()
# Add all vectors in one batch
new_kv.add_vectors(keys, vectors)
# new_kv.save_word2vec_format('onlyWordsToVec.bin', binary=True)
new_kv.most_similar('queen', restrict_vocab=25000)


[nltk_data] Downloading package words to /Users/Alex/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /Users/Alex/nltk_data...
[nltk_data]   Package brown is already up-to-date!


[('princess', 0.635676383972168),
 ('king', 0.6336469650268555),
 ('monarch', 0.5814188122749329),
 ('royal', 0.543052613735199),
 ('majesty', 0.5350356698036194),
 ('throne', 0.5097099542617798),
 ('lady', 0.5045416355133057),
 ('crown', 0.49980056285858154),
 ('consort', 0.4955049455165863),
 ('mary', 0.4903523623943329)]

In [136]:
import numpy as np
wordSpread = ['undertaker', 'heresy', 'kidney', 'dividend', 'moccasin', 'dramatics', 'commander', 'connect']
def forward_selection(words, targetLength, model):
    while len(words) < targetLength:
        # opposites = [new_kv.most_similar(negative=[word], topn=1)[0][0] for word in words]
        # print("from opposites", new_kv.most_similar(positive=opposites, topn=1)[0][0])
        print("from negatives", new_kv.most_similar(negative=words, topn=1)[0][0])
        # words.append(new_kv.most_similar(positive=opposites, topn=1)[0][0])
        words.append(new_kv.most_similar(negative=words, topn=1)[0][0])
    print(new_kv.rank_by_centrality(words))
    return words

def backwards_selection(words, targetLength, model):
    while len(words) > targetLength:
        words.remove(new_kv.rank_by_centrality(words)[0][1])
    print(new_kv.rank_by_centrality(words))
    return words

def cosine_similarity_matrix(word_list, model):
    """
    Generate a 2D array of cosine similarities between all elements in a list of words.

    :param word_list: List of words to compute the cosine similarity matrix for.
    :param model: Pre-loaded Gensim KeyedVectors model containing the word vectors.
    :return: 2D NumPy array of cosine similarities.
    """
    # Filter the list to keep only words present in the model
    valid_words = [word for word in word_list if word in model.key_to_index]
    
    # Retrieve the vectors for these words
    vectors = np.array([model[word] for word in valid_words])
    
    # Normalize the vectors to unit length
    vectors_norm = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    
    # Compute the cosine similarity matrix
    similarity_matrix = np.dot(vectors_norm, vectors_norm.T)
    
    return similarity_matrix

def back_select(word_list, model):
    """
    Remove the word with the highest average cosine similarity to the other elements in the list,
    using the cosine_similarity_matrix function for computation.

    :param word_list: List of words.
    :param model: Pre-loaded Gensim KeyedVectors model containing the word vectors.
    :return: Modified list with the word removed.
    """
    # Generate cosine similarity matrix for the word list
    sim_matrix = cosine_similarity_matrix(word_list, model)
    
    # Compute the average cosine similarity per word (excluding self-similarity)
    np.fill_diagonal(sim_matrix, 0)  # Ensure self-similarity is not considered
    avg_sim = np.mean(sim_matrix, axis=1)
    
    # Find the index of the word with the highest average similarity
    max_sim_index = np.argmax(avg_sim)
    
    # Identify the word to remove
    valid_words = [word for word in word_list if word in model.key_to_index]
    word_to_remove = valid_words[max_sim_index]
    
    print(f"Drop '{word_to_remove}'.")
    
    # Remove the identified word from the original list and return the modified list
    modified_list = [word for word in word_list if word != word_to_remove]
    return modified_list

def bidirectional_selection(words, targetLength, iterations, model):
    for i in range(iterations):
        newWord = new_kv.most_similar(negative=words, topn=1)[0][0]
        words.append(newWord)
        if(len(words) > targetLength):
            words = back_select(words, new_kv)
    return words

wordSpread = ["apple"]
wordSpread10 = forward_selection(wordSpread, 10, new_kv)

# print(bidirectional_selection(["apple", "elephant", "vacancy"], 10, 50, new_kv))



from negatives rationalistic
from negatives undersecretary
from negatives blues
from negatives chlorine
from negatives shipwreck
from negatives elect
from negatives arabesque
from negatives distance
from negatives aw
[(0.29509893, 'aw'), (0.2633828, 'distance'), (0.24080755, 'blues'), (0.23872589, 'arabesque'), (0.2085289, 'chlorine'), (0.14262965, 'elect'), (0.13942298, 'shipwreck'), (0.12546945, 'apple'), (0.123517394, 'rationalistic'), (0.10731378, 'undersecretary')]


In [81]:
import numpy as np
model = new_kv
def calculate_mean_distance(word, word_list, model):
    """Calculate the mean distance of a word from a list of words."""
    if not word_list:  # Avoid division by zero
        return float('inf')  
    distances = model.distances(word, word_list)
    return np.mean(distances)

def forward_selection(model, initial_list, n_words, vocab_limit=10000):
    """
    Perform forward selection to maximize mean distance between words in the list.
    
    :param model: KeyedVectors model
    :param initial_list: list of initial words (can be empty)
    :param n_words: target number of words in the list
    :param vocab_limit: number of top frequent words to consider from the vocabulary
    """
    selected_words = initial_list[:]
    vocabulary = [word for word, _ in model.most_similar(negative=selected_words, topn=vocab_limit)]

    while len(selected_words) < n_words and vocabulary:
        max_distance = 0
        best_candidate = None
        for candidate in vocabulary:
            mean_distance = calculate_mean_distance(candidate, selected_words, model)
            if mean_distance > max_distance:
                max_distance = mean_distance
                best_candidate = candidate
                
        if best_candidate:
            selected_words.append(best_candidate)
            vocabulary.remove(best_candidate)
            print(f"Added: {best_candidate}, Mean Distance: {max_distance}")
        else:
            break  # Stop if no suitable candidate is found

    return selected_words

def backward_selection(selected_words, model):
    """
    Perform backward selection by removing the least distinctive word from the list
    in a more elegant way.
    
    :param selected_words: The initial list of words.
    :param model: The KeyedVectors model containing the word vectors.
    :return: The modified list with one word removed.
    """
    if len(selected_words) <= 1:
        return selected_words  # Cannot remove words from a list with 1 or 0 elements.

    # Calculate the overall mean distance for the list without each word.
    mean_distances_without_word = [
        (word, calculate_mean_distance(word, selected_words[:i] + selected_words[i+1:], model))
        for i, word in enumerate(selected_words)
    ]

    # Find the word whose removal results in the highest mean distance.
    word_to_remove, _ = min(mean_distances_without_word, key=lambda x: x[1])

    # Remove the least distinctive word from the list.
    print(f"Removing: {word_to_remove}")
    selected_words.remove(word_to_remove)

    return selected_words

def calculate_mean_distances(word_list, model):
    """
    Calculate the mean distance of each word from all other words in the list.
    
    :param word_list: A list of words to calculate mean distances for.
    :param model: The KeyedVectors model containing the word vectors.
    :return: A dictionary with words as keys and their mean distances as values.
    """
    mean_distances = {}
    
    for i, word in enumerate(word_list):
        if word not in model.key_to_index:
            continue  # Skip words not in the model
        
        # Collect all other words except the current one
        other_words = word_list[:i] + word_list[i+1:]
        
        # Filter out words not in the model
        other_words = [w for w in other_words if w in model.key_to_index]
        
        if not other_words:  # If no other valid words are left, skip
            continue
        
        # Calculate distances from the current word to all other words
        distances = model.distances(word, other_words)
        
        # Compute the mean distance and store it
        mean_distances[word] = np.mean(distances)
    
    return mean_distances

# Example usage
wordSpread = ['undertaker', 'heresy', 'kidney', 'dividend', 'moccasin', 'whetstone', 'dramatics']
wordSpread = ['apple']

selected_words = forward_selection(model, wordSpread, 10)  # Target 10 words in the list
print("Final selection:", selected_words)

Added: levator, Mean Distance: 1.295461893081665
Added: woodrow, Mean Distance: 1.1436173915863037
Added: accommodation, Mean Distance: 1.102936029434204
Added: manto, Mean Distance: 1.0776101350784302
Added: mobster, Mean Distance: 1.0628013610839844
Added: gainfully, Mean Distance: 1.0435519218444824
Added: nineteen, Mean Distance: 1.0391043424606323
Added: bolivar, Mean Distance: 1.0277533531188965
Added: photochemical, Mean Distance: 1.0326476097106934
Final selection: ['apple', 'levator', 'woodrow', 'accommodation', 'manto', 'mobster', 'gainfully', 'nineteen', 'bolivar', 'photochemical']


In [148]:
from nltk.corpus import words, wordnet
import nltk
nltk.download('words')
nltk.download('wordnet')

# Load a broad list of English words
all_words = set(words.words())

def is_valid_word(word):
    """
    Check if a word is a valid Scrabble word using WordNet to attempt to exclude proper nouns.
    This function returns True if the word has at least one synset that is not tagged as a proper noun.
    """
    # Check if the word has synsets (i.e., is recognized by WordNet)
    synsets = wordnet.synsets(word)
    if not synsets:
        return False  # Word is not recognized by WordNet
    
    # Check if the word is likely not a proper noun
    # Note: This isn't foolproof, as it relies on WordNet's classification
    for synset in synsets:
        if 'noun' in synset.lexname() and not synset.name().startswith(word.lower()):
            return True  # Word has a noun meaning that is not a proper noun
    return False

# Filter words based on the criteria
filtered_words = {word for word in all_words if is_valid_word(word)}

print(f"Filtered {len(filtered_words)} words")
print(list(filtered_words)[0:10])  # Print a sample of the filtered list
words_list = ['king', 'queen', 'Henry', 'ugh', 'run', 'jump']
filtered_words = [word for word in words_list if is_valid_word(word)]

print("Filtered words:", filtered_words)


[nltk_data] Downloading package words to /Users/Alex/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Filtered 16144 words
['blitheness', 'prolongation', 'dig', 'Myxobacteriales', 'petchary', 'flak', 'sorrow', 'vibrancy', 'belligerency', 'hoariness']
Filtered words: ['king', 'queen', 'run', 'jump']


In [171]:
import numpy as np
from scipy.spatial import distance

# Load the model (assuming it's already loaded as 'model')
# model = gensim.downloader.load('glove-wiki-gigaword-300')  # For example

# Get the vector for "king"
king_vector = model.get_vector("king")

# Initialize a list to hold the closest words for each dimension
closest_words = []

# Iterate over each dimension in the "king" vector
for dim in range(len(king_vector)):
    min_distance = float('inf')  # Start with infinity as the minimum distance
    closest_word = None
    
    # Iterate over all words in the model
    for word in model.key_to_index.keys():
        if word == "king":  # Skip "king" itself
            continue
        
        # Calculate the absolute difference for the current dimension
        word_vector = model.get_vector(word)
        diff = abs(king_vector[dim] - word_vector[dim])
        
        # If this word is closer than the current closest, update the closest word
        if diff < min_distance:
            min_distance = diff
            closest_word = word
    
    # Add the closest word for this dimension to the list
    closest_words.append(closest_word)


In [204]:
import numpy as np

# Assuming 'model' is your word embedding model (e.g., KeyedVectors from Gensim)
king_vector = model.get_vector("peanut")
all_vectors = model.vectors  # All word vectors in the model

# Compute cosine similarities between 'king' and all other vectors
# Normalize vectors for cosine similarity
king_vector_norm = king_vector / np.linalg.norm(king_vector)
all_vectors_norm = all_vectors / np.linalg.norm(all_vectors, axis=1)[:, np.newaxis]

# Compute cosine similarities
cosine_similarities = np.dot(all_vectors_norm, king_vector_norm)

# Specify the distance threshold ±0.01 from a given distance (e.g., 1 for identical)
low_threshold = .4
high_threshold = .45

# Find indices where cosine similarity is within the specified threshold, excluding the exact match
# This step is crucial to exclude the vector for "king" itself, assuming it's the only perfect match
similar_indices = np.where((cosine_similarities > low_threshold) & 
                           (cosine_similarities < high_threshold))[0]  # Exclude exact match (cosine similarity = 1)

# Retrieve the corresponding words for these indices
similar_words = [model.index_to_key[index] for index in similar_indices]

similar_words


['sugar',
 'fruit',
 'corn',
 'juice',
 'cake',
 'flour',
 'garlic',
 'pie',
 'lemon',
 'salad',
 'honey',
 'fried',
 'shrimp',
 'bean',
 'vanilla',
 'syrup',
 'snack',
 'paste',
 'flavored',
 'spicy',
 'spinach',
 'popcorn',
 'almond',
 'salmonella',
 'cracker',
 'mayonnaise',
 'watermelon',
 'fudge',
 'tortilla',
 'tofu',
 'margarine',
 'avocado',
 'pistachio',
 'toffee']

In [370]:
def getPercentileWords(word, model):
    cosine_similarities = model.cosine_similarities(model.get_vector(word), model.vectors)
    sorted_indices = np.argsort(-cosine_similarities)
    percentile_indices = np.linspace(0, len(sorted_indices) / 200, 10).astype(int)
    percentile_words = [model.index_to_key[index] for index in sorted_indices[percentile_indices]]

    return percentile_words

top_n = 10000
limited_vocab = {word: model.get_vector(word) for word in list(model.key_to_index.keys())[:top_n]}

# Create a new model with this limited vocabulary
filtered_model = KeyedVectors(vector_size=model.vector_size)
filtered_model.add_vectors(list(limited_vocab.keys()), list(limited_vocab.values()))

print(getPercentileWords("bright", filtered_model)[::-1])

['lively', 'sunshine', 'colour', 'gray', 'shining', 'vivid', 'vibrant', 'purple', 'colored', 'bright']


In [470]:
def getSpreadPercentileWords(word, model:KeyedVectors, top_n:int):
    limited_vocab = {word: model.get_vector(word) for word in list(model.key_to_index.keys())[:top_n]}
    # Create a new model with this limited vocabulary
    filtered_model = KeyedVectors(vector_size=model.vector_size)
    filtered_model.add_vectors(list(limited_vocab.keys()), list(limited_vocab.values()))
    filtered_model.add_vector(word, model.get_vector(word))

    cosine_similarities = model.cosine_similarities(filtered_model.get_vector(word), filtered_model.vectors)
    sorted_indices = np.argsort(-cosine_similarities)
    # percentile_indices = np.linspace(0, len(sorted_indices) / 100, 10).astype(int)
    percentile_indices = [2**i for i in range(0, 10)]
    percentile_words = ["the"]
    chosen_word_indices = [0]
    for a in range(1, 9):
        binIndices = list(sorted_indices[percentile_indices[a]:percentile_indices[a+1]]) + chosen_word_indices
        binWords = [filtered_model.index_to_key[i] for i in binIndices]
        binDict = dict(zip(binWords, binIndices))
        binVectors = filtered_model.vectors_for_all(binWords)
        wordToAdd = binVectors.most_similar(negative=percentile_words)[0][0]
        percentile_words.append(wordToAdd)
        print(filtered_model.similarity(wordToAdd, word))
        chosen_word_indices.append(binDict[wordToAdd])
    percentile_words[0] = word

    return percentile_words

top_n = 10000
print(getSpreadPercentileWords("intricate", model, top_n))

0.5310387
0.5150255
0.43621874
0.40101203
0.34860992
0.32583195
0.26765046
0.21883716
['intricate', 'delicate', 'geometric', 'beautifully', 'weaving', 'maze', 'intrigue', 'devise', 'incomplete']


In [295]:
import numpy as np
import random

def setup_game(model):
    # 1. Choose a secret word randomly from the model's vocabulary
    secret_word = random.choice(list(model.key_to_index.keys()))
    
    # 2. Compute cosine similarities to the secret word for all words in the model
    secret_vector = model.get_vector(secret_word)
    all_vectors = model.vectors  # All word vectors in the model
    cosine_similarities = model.cosine_similarities(secret_vector, all_vectors)
    
    # 3. Sort indices by similarity (descending order)
    sorted_indices = np.argsort(-cosine_similarities)
    
    # Convert indices back to words
    sorted_words = [model.index_to_key[index] for index in sorted_indices]
    
    return secret_word, sorted_words

def model_response(model, input1, sorted_words):
    # Ensure input1 is in the sorted words list
    if input1 not in sorted_words:
        return "The word is not in the model's vocabulary."
    
    # Find the index of input1 in the sorted list
    input1_index = sorted_words.index(input1)
    
    # Calculate the new index to find a word closer to the secret word
    # but still "above" the user's guess in the sorted order.
    # Ensure the calculated index is at least 0.
    target_index = max(input1_index // 2, 0)
    
    # Retrieve the word at the new index
    response_word = sorted_words[target_index]
    
    return response_word


# Example usage
secret_word, sorted_words = setup_game(model)
print(f"Secret word selected. Start guessing!")

# Simulating a user guess
user_guess = "fish"
response = model_response(model, user_guess, sorted_words)
print(f"Model's hint: {response}")


Secret word selected. Start guessing!
Model's hint: intruding


In [361]:
def getHalfwayCloserWords(word, model):
    # Compute cosine similarities between the given word and all vectors in the model
    cosine_similarities = model.cosine_similarities(model.get_vector(word), model.vectors)
    
    # Sort indices of vectors by their cosine similarity in descending order
    sorted_indices = np.argsort(-cosine_similarities)
    
    # Initialize the list to store indices of words to be selected
    selected_indices = []
    current_index = 1024
    
    # Keep selecting words until we reach the closest word
    while current_index > 1:
        selected_indices.append(current_index)
        current_index //= 2 # Move closer to the goal word
    
    # Adjust for 0-based indexing and ensure we include the closest word
    adjusted_indices = [sorted_indices[i-1] for i in selected_indices if i > 0]
    adjusted_indices.append(sorted_indices[0])  # Ensure the closest word is included
    
    # Retrieve the words corresponding to selected indices
    halfway_closer_words = [model.index_to_key[index] for index in adjusted_indices]
    
    return halfway_closer_words

# Example usage
halfway_closer_words = getHalfwayCloserWords("rogue", model)
print(halfway_closer_words)


['hereward', 'wandering', 'feared', 'vulnerable', 'clone', 'trooper', 'militia', 'trader', 'renegade', 'covert', 'rogue']


In [354]:
from scipy.spatial import distance
import random
top_n = 10000
limited_vocab = {word: model.get_vector(word) for word in list(model.key_to_index.keys())[:top_n]}

# Create a new model with this limited vocabulary
filtered_model = KeyedVectors(vector_size=model.vector_size)
filtered_model.add_vectors(list(limited_vocab.keys()), list(limited_vocab.values()))

secret_word = random.choice(list(model.key_to_index.keys())[:5000])
def find_closest_difference(target_word, guess_word, model:KeyedVectors):
    if guess_word not in model.key_to_index or target_word not in model.key_to_index:
        return None, "Both words must be in the model's vocabulary."
    
    hint = model.most_similar_cosmul(positive = [target_word, guess_word])[7]
    return hint, "is a clue based on your guess", guess_word

# Function to provide feedback based on cosine similarity
def provide_feedback(guess, secret_word, model):
    guess_vec = model.get_vector(guess)
    secret_vec = model.get_vector(secret_word)
    cos_sim = 1 - distance.cosine(guess_vec, secret_vec)  # Cosine similarity ranges from 0 to 1
    
    # Categorize feedback based on similarity ranges
    if cos_sim > 0.75:
        feedback = "Very close!"
    elif cos_sim > 0.5:
        feedback = "Somewhat close."
    elif cos_sim > 0.25:
        feedback = "Not very close."
    else:
        feedback = "Far off."
    
    return cos_sim, feedback

# Game loop
for attempt in range(10):
    guess = input("Enter your guess: ").strip()
    if guess not in model.key_to_index:
        print("Word not in vocabulary. Try a different word.")
        continue
    
    similarity, feedback = provide_feedback(guess, secret_word, model)
    print(find_closest_difference(secret_word, guess, model))
    print(f"Feedback: {feedback} (Similarity: {similarity:.2f})")
    
    if similarity > 0.75:  # Considered a win
        print("Congratulations, you've guessed closely enough!")
        break
print("the secret word is", secret_word)


(('gift', 0.4092918634414673), 'is a clue based on your guess', 'snack')
Feedback: Far off. (Similarity: -0.02)
(('your', 0.47820284962654114), 'is a clue based on your guess', 'gift')
Feedback: Not very close. (Similarity: 0.30)
(('personal', 0.5165568590164185), 'is a clue based on your guess', 'your')
Feedback: Not very close. (Similarity: 0.36)
(('lifelong', 0.4929089844226837), 'is a clue based on your guess', 'personal')
Feedback: Not very close. (Similarity: 0.37)
(('child', 0.47831782698631287), 'is a clue based on your guess', 'marriage')
Feedback: Not very close. (Similarity: 0.33)
(('own', 0.4845896065235138), 'is a clue based on your guess', 'family')
Feedback: Not very close. (Similarity: 0.29)
(('time', 0.5375105142593384), 'is a clue based on your guess', 'own')
Feedback: Not very close. (Similarity: 0.30)
(('work', 0.5439039468765259), 'is a clue based on your guess', 'time')
Feedback: Not very close. (Similarity: 0.39)
(('doing', 0.5219708681106567), 'is a clue based o